In [1]:
import io,os,sys,json

notebook_dir = os.getcwd()
print('notebook dir: ', notebook_dir)
project_dir = os.path.dirname(notebook_dir)
print('project dir: ', project_dir)
sys.path.insert(0, project_dir)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dingdong.settings")

import django
django.setup()

def print_qs(querySet):
    from django.core import serializers
    data = serializers.serialize("json", querySet, indent=4)
    print(data)

notebook dir:  D:\workspace\python\dingdong\notebooks
project dir:  D:\workspace\python\dingdong


In [2]:
from base.models import Account
from django.contrib.auth.models import Group
from django.contrib.auth.models import Permission
from django.contrib.auth import authenticate
from django.contrib.contenttypes.models import ContentType

In [132]:
from rest_framework.request import Request
from rest_framework.test import APIRequestFactory

request_factory = APIRequestFactory()

serializer_context = {
    'request': Request(request_factory.get('/')),
}

from rest_framework import serializers
from rest_framework.renderers import JSONRenderer
# from base.api.serializers import AccountSerializer
# from base.api.serializers import GroupSerializer

In [90]:
class SimpleDictField(serializers.RelatedField):
    def to_representation(self, value):
        obj = {}
        for field in type(value)._meta.get_fields():
            if field.is_relation:
                continue
            obj[field.name] = getattr(value, field.name)
        return obj

In [91]:
class PermissionSerializer(serializers.ModelSerializer):

    # content_type = ContentTypeSerializer()
    app_label = serializers.CharField(source='content_type.app_label', read_only=True)
    model = serializers.CharField(source='content_type.model', read_only=True)
    content_type_id = serializers.IntegerField(source='content_type.id', read_only=True)

    class Meta:
        model = Permission
        fields = ('codename', 'app_label', 'model', 'content_type_id', 'name')
        # fields = '__all__'

class GroupSerializer(serializers.HyperlinkedModelSerializer):

    permissions = PermissionSerializer(many=True, read_only=True)
    accounts = SimpleDictField(source="user_set", many=True, read_only=True)

    class Meta:
        model = Group
        fields = ('url', 'id', 'name', 'permissions', 'accounts')
        # fields = ('url', 'name')

In [93]:
class AccountSerializer(serializers.HyperlinkedModelSerializer):

    groups = GroupSerializer(many=True, read_only=True)
    user_permissions = PermissionSerializer(many=True, read_only=True)

    class Meta:
        model = Account
        fields = ('url', 'username', 'email', 'first_name', 'last_name', 'groups', 'user_permissions')
#         fields = ('url', 'username', 'email', 'groups')

serializer = AccountSerializer()
print(repr(serializer))

AccountSerializer():
    url = HyperlinkedIdentityField(view_name='account-detail')
    username = CharField(help_text='Required. 150 characters or fewer. Letters, digits and @/./+/-/_ only.', max_length=150, validators=[<django.contrib.auth.validators.UnicodeUsernameValidator object>, <UniqueValidator(queryset=Account.objects.all())>])
    email = EmailField(allow_blank=True, label='Email address', max_length=254, required=False)
    first_name = CharField(allow_blank=True, max_length=30, required=False)
    last_name = CharField(allow_blank=True, max_length=150, required=False)
    groups = GroupSerializer(many=True, read_only=True):
        url = HyperlinkedIdentityField(view_name='group-detail')
        id = IntegerField(label='ID', read_only=True)
        name = CharField(max_length=80, validators=[<UniqueValidator(queryset=Group.objects.all())>])
        permissions = PermissionSerializer(many=True, read_only=True):
            codename = CharField(max_length=100)
            app

In [94]:
serializer = AccountSerializer(Account.objects.get(username='ada'), context=serializer_context)
parsed = json.loads(JSONRenderer().render(serializer.data).decode("utf-8"))
print(json.dumps(parsed, indent=4, sort_keys=True))

{
    "email": "ada@glittergroupcn.com",
    "first_name": "",
    "groups": [
        {
            "accounts": [
                {
                    "date_joined": "2019-03-28T14:51:05.921391Z",
                    "email": "ada@glittergroupcn.com",
                    "first_name": "",
                    "id": 2,
                    "is_active": true,
                    "is_staff": false,
                    "is_superuser": false,
                    "last_login": null,
                    "last_name": "",
                    "password": "",
                    "username": "ada"
                }
            ],
            "id": 1,
            "name": "salesperson",
            "permissions": [
                {
                    "app_label": "base",
                    "codename": "view_customer",
                    "content_type_id": 13,
                    "model": "customer",
                    "name": "Can view customer"
                },
                {
            